# Workflow<br>
0.Import the important libraries<br>
1.Prepare the dataset<br>
2.Build the model<br>
3.Pick the loss function and optimizer<br>
4.Build the train and evaluate loop<br>
5.Train and Evaluate model<br>
6.Test on unseen datasets<br>

In [1]:
!pip uninstall -y torch torchvision torchtext
!pip install torch==2.3.0 torchtext==0.18.0

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 839.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
!pip install datasets evaluate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you hav

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import spacy
import datasets
import torchtext
import tqdm
import evaluate

In [4]:
train_data,val_data,test_data = datasets.load_dataset("bentrevett/multi30k",split=["train","validation","test"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

val.jsonl:   0%|          | 0.00/164k [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/156k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
train_data

Dataset({
    features: ['en', 'de'],
    num_rows: 29000
})

In [6]:
test_data

Dataset({
    features: ['en', 'de'],
    num_rows: 1000
})

In [7]:
val_data

Dataset({
    features: ['en', 'de'],
    num_rows: 1014
})

tokenize data

In [8]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 80.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
en_nlp=spacy.load("en_core_web_sm")
de_nlp=spacy.load("de_core_news_sm")

In [10]:
def tokenize_text(text,en_nlp,de_nlp,max_length,lower,sos,eos):
  en_tokens = [token.text for token in en_nlp.tokenizer(text["en"])][:max_length]
  de_tokens = [token.text for token in de_nlp.tokenizer(text["de"])][:max_length]

  if lower:
    en_tokens = [token.lower() for token in en_tokens]
    de_tokens = [token.lower() for token in de_tokens]

  en_tokens = [sos] + en_tokens + [eos]
  de_tokens = [sos] + de_tokens + [eos]

  return {
      "en_tokens":en_tokens,
      "de_tokens":de_tokens
  }

In [11]:
max_length=1000
lower=True
sos_token="<sos>"
eos_token="<eos>"
fn_kwargs={
    "en_nlp":en_nlp,
    "de_nlp":de_nlp,
    "max_length":max_length,
    "lower":lower,
    "sos":sos_token,
    "eos":eos_token
}

In [12]:
train_data = train_data.map(tokenize_text,fn_kwargs=fn_kwargs)
val_data = val_data.map(tokenize_text,fn_kwargs=fn_kwargs)
test_data = val_data.map(tokenize_text,fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

creating vocabulary

In [13]:
from torchtext.vocab import build_vocab_from_iterator

/usr/local/lib/python3.11/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.11/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [14]:
min_freq=2
unk_token="<unk>"
pad_token="<pad>"
special_tokens= [sos_token,eos_token,unk_token,pad_token]

In [15]:
en_vocab = build_vocab_from_iterator(train_data["en_tokens"],min_freq=min_freq,specials=special_tokens)
de_vocab = build_vocab_from_iterator(train_data["de_tokens"],min_freq=min_freq,specials=special_tokens)

In [16]:
assert en_vocab[unk_token] == de_vocab[unk_token]
assert en_vocab[pad_token] == de_vocab[pad_token]

unk_idx = en_vocab[unk_token]
pad_idx = en_vocab[pad_token]

In [17]:
en_vocab.set_default_index(unk_idx)
de_vocab.set_default_index(unk_idx)

Numercalize the data

In [18]:
def numericalize_data(data,en_vocab,de_vocab):
  en_idxs = en_vocab.lookup_indices(data["en_tokens"])
  de_idxs = de_vocab.lookup_indices(data["de_tokens"])
  return{
      "en_idxs":en_idxs,
      "de_idxs":de_idxs
  }


In [19]:
fn_kwargs={
    "en_vocab":en_vocab,
    "de_vocab":de_vocab
}

In [20]:
train_data = train_data.map(numericalize_data,fn_kwargs=fn_kwargs)
val_data = val_data.map(numericalize_data,fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_data,fn_kwargs=fn_kwargs)

Map:   0%|          | 0/29000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

In [21]:
train_data

Dataset({
    features: ['en', 'de', 'en_tokens', 'de_tokens', 'en_idxs', 'de_idxs'],
    num_rows: 29000
})

Converting into tensors

In [22]:
data_type="torch"
format_columns=["en_idxs","de_idxs"]

In [23]:
train_data = train_data.with_format(type=data_type,columns=format_columns,output_all_columns=True)
test_data = test_data.with_format(type=data_type,columns=format_columns,output_all_columns=True)
val_data = val_data.with_format(type=data_type,columns=format_columns,output_all_columns=True)

In [24]:
type(train_data[0]["en_idxs"])

torch.Tensor

converting into batch

In [25]:
def get_collate_fn(pad_index):
  def collate_fn(batch): # batch is a list of dictionaries
    # Each item in 'batch' is a dictionary like {'en_idxs': tensor(...), 'de_idxs': tensor(...), ...}
    list_of_en_idxs = [example['en_idxs'] for example in batch]
    list_of_de_idxs = [example['de_idxs'] for example in batch]

    # Pad these lists of tensors
    # batch_first=True will make the output (batch_size, max_seq_len)
    padded_en_idxs = nn.utils.rnn.pad_sequence(list_of_en_idxs, batch_first=True, padding_value=pad_index)
    padded_de_idxs = nn.utils.rnn.pad_sequence(list_of_de_idxs, batch_first=True, padding_value=pad_index)

    return {
        "en_idxs": padded_en_idxs, # Target sequence (English)
        "de_idxs": padded_de_idxs  # Source sequence (German)
    }
  return collate_fn

In [26]:
def get_data_loader(data,batch_size,pad_idx,shuffle=False,):
  collate_fn = get_collate_fn(pad_idx)
  dataloader = torch.utils.data.DataLoader(
      data,
      batch_size,
      collate_fn=collate_fn,
      shuffle=shuffle
  )
  return dataloader

In [27]:
batch_size = 128

In [28]:
train_data_loader = get_data_loader(train_data,batch_size=batch_size,pad_idx=pad_idx,shuffle=True)
val_data_loader = get_data_loader(test_data,batch_size=batch_size,pad_idx=pad_idx)
test_data_loader = get_data_loader(test_data,batch_size=batch_size,pad_idx=pad_idx)

In [29]:
# train_data_loader.di

### Model Building

In [30]:
class Encoder(nn.Module):
  def __init__(self,input_dim,embedding_dim,hidden_dim,n_layers,dropout): # Corrected 'droupout' to 'dropout'
    super().__init__()
    self.input_dim = input_dim
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim # Added
    self.n_layers = n_layers   # Added
    self.dropout_rate = dropout

    self.embedding = nn.Embedding(input_dim, embedding_dim=embedding_dim)
    # LSTM input_size should be embedding_dim, not input_dim (vocab size)
    self.lstm = nn.LSTM(embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout)
    self.dropout_layer = nn.Dropout(dropout) # Renamed from self.droupout for clarity

  def forward(self, src): # src is (seq_len, batch_size)
    # Pass source language tokens through embedding layer
    embedded = self.dropout_layer(self.embedding(src))
    # embedded shape: (seq_len, batch_size, embedding_dim)
    # Pass embeddings through LSTM
    # outputs from LSTM are all hidden states, (hidden, cell) are the final hidden and cell states
    outputs, (hidden, cell) = self.lstm(embedded)
    return (hidden, cell)

In [31]:
class Decoder(nn.Module):
  # Removed 'input_dim' from constructor as it's not used;
  # 'output_dim' (target vocab size) is used for embedding.
  def __init__(self, output_dim, embedding_dim, hidden_dim, n_layers, dropout):
    super().__init__()
    self.output_dim = output_dim # This is the target vocabulary size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim # Added
    self.n_layers = n_layers   # Added
    self.dropout_rate = dropout

    self.embedding = nn.Embedding(output_dim, embedding_dim=embedding_dim)
    # LSTM input_size should be embedding_dim
    self.lstm = nn.LSTM(embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, dropout=dropout)
    self.dropout_layer = nn.Dropout(dropout) # Renamed from self.dropout for clarity
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x, hidden, cell): # x is current input token, shape (batch_size)
    x = x.unsqueeze(0) # Reshape x to (1, batch_size) for sequence length of 1
    embed = self.dropout_layer(self.embedding(x))
    # embed shape: (1, batch_size, embedding_dim)
    output, (hidden, cell) = self.lstm(embed, (hidden, cell))
    # output shape: (1, batch_size, hidden_dim)
    prediction = self.fc(output.squeeze(0)) # Squeeze seq_len dim -> (batch_size, hidden_dim) -> (batch_size, output_dim)
    # prediction shape: (batch_size, output_dim)
    return prediction, hidden, cell

In [32]:
class Seq2Seq(nn.Module):
  def __init__(self,encoder,decoder,device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device=device

    assert encoder.hidden_dim == decoder.hidden_dim, "Hidden dimension of encoder and decoder must be same"
    assert encoder.n_layers == decoder.n_layers, "n_layers of encoder and decoder must be same"

  def forward(self,src,target,teacher_forcing_ratio):
    # src: (batch_size, src_len) from DataLoader
    # target: (batch_size, trg_len) from DataLoader
    # Permute to (seq_len, batch_size) for Encoder/Decoder internals
    src = src.permute(1,0)
    target = target.permute(1,0)

    target_length = target.shape[0] # trg_len
    batch_size = target.shape[1]
    target_vocab_size = self.decoder.output_dim # Accessing self.decoder.output_dim

    outputs = torch.zeros(target_length,batch_size,target_vocab_size).to(self.device)

    hidden,cell = self.encoder(src) # src is (src_len, batch_size)
    # Initial input to the decoder is the <sos> token for each sequence in the batch
    decoder_input = target[0,:] # Shape: (batch_size)

    for t in range(1,target_length): # Iterate from the first actual token up to the end
      # decoder_input shape: (batch_size)
      # hidden, cell shapes: (n_layers, batch_size, hidden_dim)
      output,hidden,cell = self.decoder(decoder_input,hidden,cell)
      # output shape from decoder: (batch_size, output_dim)
      outputs[t] = output # Store prediction for this timestep
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1) # Get most likely next token_id, shape (batch_size)
      # If teacher forcing, next input is actual next token in target sequence
      # Otherwise, next input is the token predicted by the decoder
      decoder_input = target[t] if teacher_force else top1

    return outputs

Training the model

model-initialization

In [33]:
input_dim = len(de_vocab) # Source (German) vocabulary size
output_dim = len(en_vocab) # Target (English) vocabulary size
encoder_embedding_dim = 256
decoder_embedding_dim = 256
hidden_dim = 512
n_layers=2
encoder_dropout=0.5 # Corrected typo from droupout
decoder_dropout=0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder = Encoder(
    input_dim=input_dim, # German vocab size
    embedding_dim=encoder_embedding_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    dropout=encoder_dropout # Corrected typo
)

decoder = Decoder(
    # input_dim (German vocab size) is removed from Decoder's constructor
    output_dim=output_dim, # English vocab size
    embedding_dim=decoder_embedding_dim,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    dropout=decoder_dropout
)

In [34]:
model = Seq2Seq(encoder=encoder,decoder=decoder,device=device)
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout_layer): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (lstm): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout_layer): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=512, out_features=5893, bias=True)
  )
)

count number of parameter

In [35]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f"The model has {count_parameters(model):,} trainable parameters")

The model has 13,898,501 trainable parameters


Picking loss function and optimizer

In [36]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [37]:
optimizer = optim.Adam(model.parameters(),)

Defining training fn and evalutaion fn

In [38]:
def train_fn(
    model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device
):
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(data_loader):
        src = batch["de_idxs"].to(device) # Changed "de_ids" to "de_idxs"
        trg = batch["en_idxs"].to(device) # Changed "en_ids" to "en_idxs"
        # src = [src length, batch size] (after permute in model.forward)
        # trg = [trg length, batch size] (after permute in model.forward)
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
        # output = [trg length, batch size, trg vocab size]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        # output = [(trg length - 1) * batch size, trg vocab size]
        trg = trg.permute(1,0)[1:].reshape(-1) # Correctly permute trg before slicing and viewing for loss
        # trg = [(trg length - 1) * batch size]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [39]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["de_idxs"].to(device) # Changed "de_ids" to "de_idxs"
            trg = batch["en_idxs"].to(device) # Changed "en_ids" to "en_idxs"
            # src = [src length, batch size] (after permute in model.forward)
            # trg = [trg length, batch size] (after permute in model.forward)
            output = model(src, trg, 0)  # turn off teacher forcing
            # output = [trg length, batch size, trg vocab size]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            # output = [(trg length - 1) * batch size, trg vocab size]
            trg = trg.permute(1,0)[1:].reshape(-1) # Correctly permute trg before slicing and viewing for loss
            # trg = [(trg length - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [40]:
n_epochs = 10
clip = 1.0
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm.tqdm(range(n_epochs)):
    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )
    valid_loss = evaluate_fn(
        model,
        val_data_loader,
        criterion,
        device,
    )
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), "model.pth")
    print(f"\tTrain Loss: {train_loss:7.3f} | Train PPL: {np.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:7.3f} | Valid PPL: {np.exp(valid_loss):7.3f}")

 10%|█         | 1/10 [00:44<06:40, 44.51s/it]

	Train Loss:   5.012 | Train PPL: 150.131
	Valid Loss:   4.845 | Valid PPL: 127.149


 20%|██        | 2/10 [01:30<06:02, 45.30s/it]

	Train Loss:   4.448 | Train PPL:  85.462
	Valid Loss:   4.585 | Valid PPL:  97.965


 30%|███       | 3/10 [02:15<05:15, 45.09s/it]

...

	Train Loss:   3.321 | Train PPL:  27.680
	Valid Loss:   3.878 | Valid PPL:  48.345


 90%|█████████ | 9/10 [06:47<00:45, 45.35s/it]

	Train Loss:   3.225 | Train PPL:  25.145
	Valid Loss:   3.881 | Valid PPL:  48.465


100%|██████████| 10/10 [07:32<00:00, 45.26s/it]

	Train Loss:   3.126 | Train PPL:  22.789
	Valid Loss:   3.833 | Valid PPL:  46.196


In [41]:
model.load_state_dict(torch.load("model.pth"))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

| Test Loss: 3.833 | Test PPL:  46.196 |


In [42]:
def translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            tokens = [token.text for token in de_nlp.tokenizer(sentence)]
        else:
            tokens = [token for token in sentence]
        if lower:
            tokens = [token.lower() for token in tokens]
        tokens = [sos_token] + tokens + [eos_token]
        ids = de_vocab.lookup_indices(tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)
        hidden, cell = model.encoder(tensor)
        inputs = en_vocab.lookup_indices([sos_token])
        for _ in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, cell = model.decoder(inputs_tensor, hidden, cell)
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == en_vocab[eos_token]:
                break
        tokens = en_vocab.lookup_tokens(inputs)
    return tokens

In [43]:
sentence = test_data[0]["de"]
expected_translation = test_data[0]["en"]

sentence, expected_translation

('Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen',
 'A group of men are loading cotton onto a truck')

In [44]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [45]:
translation

['<sos>', 'a', 'group', 'of', 'men', 'are', 'a', 'a', 'a', '.', '<eos>']

In [46]:
sentence = "Ein Mann sitzt auf einer Bank."

In [47]:
translation = translate_sentence(
    sentence,
    model,
    en_nlp,
    de_nlp,
    en_vocab,
    de_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

In [48]:
translation

['<sos>', 'a', 'man', 'sits', 'on', 'a', 'bench', '.', '<eos>']

In [49]:
translations = [
    translate_sentence(
        example["de"],
        model,
        en_nlp,
        de_nlp,
        en_vocab,
        de_vocab,
        lower,
        sos_token,
        eos_token,
        device,
    )
    for example in tqdm.tqdm(test_data)
]

100%|██████████| 1014/1014 [00:08<00:00, 120.26it/s]


In [50]:
bleu = evaluate.load("bleu")

In [51]:
predictions = [" ".join(translation[1:-1]) for translation in translations]

references = [[example["en"]] for example in test_data]

In [52]:
predictions[0], references[0]

('a group of men are a a a .',
 ['A group of men are loading cotton onto a truck'])

In [53]:
def get_tokenizer_fn(nlp, lower):
    def tokenizer_fn(s):
        tokens = [token.text for token in nlp.tokenizer(s)]
        if lower:
            tokens = [token.lower() for token in tokens]
        return tokens

    return tokenizer_fn

In [55]:
tokenizer_fn = get_tokenizer_fn(en_nlp, lower)

In [56]:
tokenizer_fn(predictions[0]), tokenizer_fn(references[0][0])

(['a', 'group', 'of', 'men', 'are', 'a', 'a', 'a', '.'],
 ['a', 'group', 'of', 'men', 'are', 'loading', 'cotton', 'onto', 'a', 'truck'])

In [57]:
results = bleu.compute(
    predictions=predictions, references=references, tokenizer=tokenizer_fn
)

In [58]:
results

{'bleu': 0.12967539082214283,
 'precisions': [0.47855787476280837,
  0.1875668119398076,
  0.08612182650040369,
  0.03947498272969505],
 'brevity_penalty': 0.9811290943791274,
 'length_ratio': 0.9813049307314167,
 'translation_length': 13175,
 'reference_length': 13426}